In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm
from statsmodels.tsa.stattools import adfuller

# Load dataset
data = pd.read_csv('../datasets/CropSDEData/METEO_DEKADS_NUTS2_NL.csv')

# Feature Selection
target = 'PREC'

# Drop rows with missing values
data = data.dropna(subset=[target])

# Prepare data
y = data[target].values

# Ensure stationarity
def check_stationarity(y):
    result = adfuller(y)
    return result[1]  # p-value of ADF test

if check_stationarity(y) > 0.05:
    print("Data is non-stationary. Applying differencing...")
    y = np.diff(y)

# Method of Moments (MoM) for Vasicek Model
def vasicek_mom(y):
    dt = 1  # Assuming daily intervals
    dy = y[1:] - y[:-1]
    
    # Estimate parameters
    sigma_mom = np.sqrt(np.var(dy) / dt)
    b_mom = np.mean(y)
    a_mom = -np.mean(dy) / np.mean(y[:-1] - b_mom)

    return a_mom, b_mom, sigma_mom

# Maximum Likelihood Estimation (MLE) for Vasicek Model
def vasicek_mle(params, data):
    a, b, sigma = params
    dt = 1  # Assuming daily intervals
    X = data

    # Vasicek model residuals
    residuals = X[1:] - (X[:-1] + a * (b - X[:-1]) * dt)
    
    # Log likelihood
    log_likelihood = -np.sum(0.5 * np.log(2 * np.pi * sigma**2 * dt) + (residuals**2 / (2 * sigma**2 * dt)))
    return -log_likelihood  # Negative for minimization

# Least Squares (LSQ) for Vasicek Model
def vasicek_lsq(params, data):
    a, b, sigma = params
    dt = 1  # Assuming daily intervals
    X = data

    # Predicted values using the Vasicek model
    predicted = X[:-1] + a * (b - X[:-1]) * dt

    # Residual sum of squares
    residuals = X[1:] - predicted
    return np.sum(residuals**2)

# Bayesian Estimation using MAP (Maximum A Posteriori)
def vasicek_log_posterior(params, data):
    a, b, sigma = params
    dt = 1  # Assuming daily intervals
    X = data

    # Priors
    prior_a = norm.logpdf(a, loc=0.5, scale=0.5)
    prior_b = norm.logpdf(b, loc=np.mean(data), scale=np.std(data))
    prior_sigma = norm.logpdf(sigma, loc=1, scale=0.5)

    log_prior = prior_a + prior_b + prior_sigma

    # Likelihood
    residuals = X[1:] - (X[:-1] + a * (b - X[:-1]) * dt)
    log_likelihood = -np.sum(0.5 * np.log(2 * np.pi * sigma**2 * dt) + (residuals**2 / (2 * sigma**2 * dt)))

    return log_prior + log_likelihood  # Posterior = Prior + Likelihood

def vasicek_bayesian_map(data, initial_guess):
    bounds = [(1e-5, None), (None, None), (1e-5, None)]
    res_map = minimize(
        lambda params: -vasicek_log_posterior(params, data),  # Negative log-posterior for minimization
        initial_guess,
        method='L-BFGS-B',
        bounds=bounds
    )
    return res_map

# Method of Moments (MoM)
a_mom, b_mom, sigma_mom = vasicek_mom(y)
print("\nEstimated Vasicek Parameters using Method of Moments:")
print(f"Alpha (a): {a_mom}, Beta (b): {b_mom}, Sigma: {sigma_mom}")

# Maximum Likelihood Estimation (MLE)
initial_guess = [a_mom, b_mom, sigma_mom]
res_mle = minimize(vasicek_mle, initial_guess, args=(y,), method='L-BFGS-B',
                   bounds=[(1e-5, None), (None, None), (1e-5, None)])

if res_mle.success:
    a_mle, b_mle, sigma_mle = res_mle.x
    print("\nEstimated Vasicek Parameters using Maximum Likelihood Estimation (MLE):")
    print(f"Alpha (a): {a_mle}, Beta (b): {b_mle}, Sigma: {sigma_mle}")
else:
    print("\nMLE failed to converge.")
    print(f"Message: {res_mle.message}")

# Bayesian Estimation with MAP
print("\nRunning Bayesian Estimation Using MAP...")
res_map = vasicek_bayesian_map(y, initial_guess)
if res_map.success:
    a_map, b_map, sigma_map = res_map.x
    print("\nEstimated Vasicek Parameters using Bayesian Estimation (MAP):")
    print(f"Alpha (a): {a_map}, Beta (b): {b_map}, Sigma: {sigma_map}")
else:
    print("\nMAP Estimation failed to converge.")
    print(f"Message: {res_map.message}")


Estimated Vasicek Parameters using Method of Moments:
Alpha (a): 0.42410679424415204, Beta (b): 1.9206313657407403, Sigma: 2.230060821066932

Estimated Vasicek Parameters using Maximum Likelihood Estimation (MLE):
Alpha (a): 0.9685380046177161, Beta (b): 1.9206229108630928, Sigma: 1.6015054584586133

Running Bayesian Estimation Using MAP...

Estimated Vasicek Parameters using Bayesian Estimation (MAP):
Alpha (a): 0.9684296981217096, Beta (b): 1.9206229242636266, Sigma: 1.6013269350752337
